In [1]:
import os
# os.chdir('./net2neuro')
from tqdm import tqdm
import pickle
import mne
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
# from simpleconv import SimpleConv
import torch.optim as optim
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

In [2]:
%load_ext autoreload
%autoreload 2

In [2]:
base_dir = '/scratch/jd5697/cv_project'

In [27]:
embeddings_train_fmri_subset = np.load(os.path.join(base_dir, 'image_embeddings_vit_train_fmri_subset.npy'), allow_pickle=True).item()
embeddings_test_fmri_subset = np.load(os.path.join(base_dir, 'image_embeddings_vit_test_fmri_subset.npy'), allow_pickle=True).item()
embeddings_test_small_fmri_subset = np.load(os.path.join(base_dir, 'image_embeddings_vit_test_small_fmri_subset.npy'), allow_pickle=True).item()

print(len(embeddings_train_fmri_subset), len(embeddings_test_fmri_subset), len(embeddings_test_small_fmri_subset))

6240 2400 100


In [5]:
n_participants = 4

In [34]:
with open('valid_epochs_all_train_resplit.pickle', 'rb') as f:
    epochs_train = pickle.load(f)

with open('valid_epochs_all_test_resplit.pickle', 'rb') as f:
    epochs_test = pickle.load(f)

with open('valid_epochs_small_test_resplit.pickle', 'rb') as f:
    epochs_test_small = pickle.load(f)

print(len(epochs_train), len(epochs_test), len(epochs_test_small))

79392 9600 800


In [35]:
epochs_train.metadata['embedding_exists'] = epochs_train.metadata['image_path'].apply(lambda x: x in embeddings_train_fmri_subset)
epochs_train = epochs_train[epochs_train.metadata['embedding_exists']]
epochs_train.metadata.drop(columns=['embedding_exists'], inplace=True)
epochs_train.metadata.reset_index(drop=True, inplace=True)

print(len(epochs_train), len(epochs_train) // n_participants)

24960 6240


In [36]:
epochs_test.metadata['embedding_exists'] = epochs_test.metadata['image_path'].apply(lambda x: x in embeddings_test_fmri_subset)
epochs_test = epochs_test[epochs_test.metadata['embedding_exists']]
epochs_test.metadata.drop(columns=['embedding_exists'], inplace=True)
epochs_test.metadata.reset_index(drop=True, inplace=True)

print(len(epochs_test), len(epochs_test) // n_participants)

9600 2400


In [37]:
epochs_test_small.metadata['embedding_exists'] = epochs_test_small.metadata['image_path'].apply(lambda x: x in embeddings_test_small_fmri_subset)
epochs_test_small = epochs_test_small[epochs_test_small.metadata['embedding_exists']]
epochs_test_small.metadata.drop(columns=['embedding_exists'], inplace=True)
epochs_test_small.metadata.reset_index(drop=True, inplace=True)

print(len(epochs_test_small), len(epochs_test_small) // n_participants)

400 100


In [38]:
with open('valid_epochs_all_train_fmri_subset.pickle', 'wb') as f:
    pickle.dump(epochs_train, f, protocol=pickle.HIGHEST_PROTOCOL)

In [39]:
with open('valid_epochs_all_test_fmri_subset.pickle', 'wb') as f:
    pickle.dump(epochs_test, f, protocol=pickle.HIGHEST_PROTOCOL)

In [40]:
with open('valid_epochs_all_test_small_fmri_subset.pickle', 'wb') as f:
    pickle.dump(epochs_test_small, f, protocol=pickle.HIGHEST_PROTOCOL)